In [ ]:
#import libraries
from landlab import RasterModelGrid
from landlab.components import FlowAccumulator, ChannelProfiler
from landlab.components import FastscapeEroder, LinearDiffuser #employs SPIM
from landlab.grid.create_network import network_grid_from_raster
import matplotlib.pyplot as plt
import numpy as np

#LEM parameters
K = 0.00001
U = 0.001
D = 0.1

#define landlab raster grid
rows = 40
cols = 25
dx = 500.

area_threshold = dx*dx*25.

mg = RasterModelGrid((rows,cols),dx)

#set random seed
np.random.seed(123)

#set topography
mg.add_zeros('topographic__elevation',at='node')
S0 = 0.015
for i in range(0,rows-1):
    for j in range(1,cols-1):
        mg.at_node['topographic__elevation'].reshape(mg.shape)[i,j] = float(i) * dx * S0 + np.random.random() * 100.

#set river manually, area = 1 is river, area = 0 is hillslope
mg.add_zeros('boundary',at='node')
mg.at_node['boundary'].reshape(mg.shape)[:,0] = 1
mg.at_node['boundary'].reshape(mg.shape)[:,-1] = 1
mg.at_node['boundary'].reshape(mg.shape)[-1,:] = 1

#anything that is not a river is a nan
mg.at_node['topographic__elevation'][mg.at_node['boundary'] == 1] = np.nan

#anything that is a nan is closed boundary
mg.status_at_node[np.isnan(mg.at_node['topographic__elevation'])] = 4

#lowest elevation is the outlet
mg.status_at_node[mg.at_node['topographic__elevation']==0.0] = 1

#run flow accumulator
flow = FlowAccumulator(mg, flow_director='D8')

#fluvial erosion
erode = FastscapeEroder(mg, K_sp = K)

#hillslope diffusion
diffuse = LinearDiffuser(mg, linear_diffusivity = D)

#running the model
dt = 1000. #time step
for t in range(0,2000):
    flow.run_one_step()
    erode.run_one_step(dt)
    diffuse.run_one_step(dt)
    mg.at_node['topographic__elevation'][mg.core_nodes] += U * dt

#plot elevation network
plt.figure(1)
plt.imshow(np.flipud(mg.at_node['topographic__elevation'].reshape(mg.shape)))   

#plot drainagea area
plt.figure(2)
plt.imshow(np.flipud(mg.at_node['drainage_area'].reshape(mg.shape)))  
      
#run channel profiler
profiler = ChannelProfiler(mg,number_of_watersheds=1,minimum_channel_threshold=area_threshold,main_channel_only=False)
profiler.run_one_step()      

#plot channel profiles
plt.figure(3)
outlets = list(profiler.data_structure.keys()) #outlets from channel profiler
for outlet in outlets: #cycle thru outlets
    segments = list(profiler.data_structure[outlet].keys()) #segments for that watershed
    for segment in segments: #cycle thru segments
        ids = profiler.data_structure[outlet][segment]['ids'] #ids (index) of each segment
        distance = profiler.data_structure[outlet][segment]['distances'] #distance downstream of each segment node
        eta = mg.at_node['topographic__elevation'][ids] #elevation of each segment node
        plt.scatter(distance,eta,alpha=0.5) #plot profile

#make raster grid into network grid
network_grid = network_grid_from_raster(mg,minimum_channel_threshold=area_threshold,include=["drainage_area", "topographic__elevation"])

#plotting the network grid
plt.figure(4)
#plot all nodes
plt.scatter(network_grid.x_of_node,network_grid.y_of_node,color='k')

#label nodes with elevation
# for i in range(0,network_grid.number_of_nodes):
#     plt.text(network_grid.x_of_node[i],network_grid.y_of_node[i],' n'+str(i)+',' +str(network_grid.at_node['topographic__elevation'][i]) ,va='center',ha='left')
    
#plot the links
for i in range(0,network_grid.number_of_links):
    i1 = network_grid.nodes_at_link[i][0]
    i2 = network_grid.nodes_at_link[i][1]
    x1,y1 = network_grid.x_of_node[i1],network_grid.y_of_node[i1]
    x2,y2 = network_grid.x_of_node[i2],network_grid.y_of_node[i2]
    plt.plot([x1,x2],[y1,y2],color='r')
plt.gca().set_aspect('equal')

In [ ]:
profiler.data_structure
Qlist = [25.,10.,15.,10.,5.] #get from landlab. NILAY CHANGE THIS!!! 
#maybe drainage area * precipt value that gets to the same order of magnitude.

nx_list = [6,6,6,6,3] #get from landlab. NILAY CHANGE THIS!!! make sure this is in the right order.
#number of segments comes from landlab channel profiler.

#initalize lists to hold data #nx_list = []
upstream_segment_list = []
downstream_segment_list = []
distance_list = []
elevation_list = []

#run thru segments in the outlets
for outlet in outlets: #cycle thru outlets
    segments = list(profiler.data_structure[outlet].keys()) #segments for that watershed
    for i, segment_A in enumerate(segments): #cycle thru segments
        ids = profiler.data_structure[outlet][segment_A]['ids']
        distance_list += [profiler.data_structure[outlet][segment_A]['distances']] #distance downstream of each segment node]
        elevation_list += [mg.at_node['topographic__elevation'][ids]] #distance downstream of each segment node]
        upstream_temp = []
        downstream_temp = []
        for j, segment_B in enumerate(segments): #cycle thru segments
            if segment_A[1] == segment_B[0]:
                upstream_temp += [j]
            if segment_A[0] == segment_B[1]:
                downstream_temp = [j]
                
        upstream_segment_list += [upstream_temp]
        downstream_segment_list += [downstream_temp]

print (Qlist)
print (upstream_segment_list)
print (downstream_segment_list)
print ('meow')
for i in range(0,len(distance_list)):
    print (distance_list[i])
print ('meow')
for i in range(0,len(elevation_list)):
    print (elevation_list[i])
print ('done')

In [ ]:
len(downstream_segment_list) #gives you number of segments

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import grlp
# For on-the-fly retries, with Python 2/3/+?? support
try:
    from importlib import reload
except ImportError:
    pass
reload(grlp)

# Choose whether to draw plots interactively
#plt.ion()
plt.ioff()

S0 = 0.015
P_xB = 0.2
z1 = 0 # Interesting -- my RHS always has to be 0 (not quite right)

dt = 3.15E7 # time step

nseg = 5 # number of segments

segments = []
for i in range(nseg):
    segments.append(grlp.LongProfile()) # 5 lp classes

# Qlist = [5., 10., 15., 10., 25.]
# upstream_segment_list = [[], [], [0,1], [], [2,3]]
# downstream_segment_list = [[2], [2], [4], [4], []] --outlet segment must be the latest in the list?--

i = 0
switch = 0
for lp in segments:
    lp.set_ID(i)
    lp.set_upstream_segment_IDs(upstream_segment_list[i])
    lp.set_downstream_segment_IDs(downstream_segment_list[i])
    lp.set_intermittency(1)
    lp.basic_constants()
    lp.bedload_lumped_constants()
    lp.set_hydrologic_constants()
    # Local or global x -- really just care about diffs for solver
    dx=500.
    nx = nx_list[i]

    #Changed so distance values are read from the landlab channel profiler, x going up means going downstream
    if len(lp.downstream_segment_IDs) > 0:
        _x = distance_list[i][::-1][:-1]
        dx_ds = _x[-2] - _x[-1]
        dx_us = _x[0] - _x[1]
        x_ext = - np.hstack((_x[0]+dx_us, _x, _x[-1]-dx_ds))
        lp.set_x(x_ext = x_ext)
    else:
        _x = distance_list[i][::-1]
        dx_ds = _x[-2] - _x[-1]
        dx_us = _x[0] - _x[1]
        x_ext =  - np.hstack((_x[0]+dx_us, _x, _x[-1]-dx_ds))
        lp.set_x(x_ext = x_ext)
        
    #Changed so elevation values are read from the landlab channel profiler
    if len(lp.downstream_segment_IDs) > 0:
        lp.set_z(z = elevation_list[i][::-1][:-1])
    else:
        lp.set_z(z = elevation_list[i][::-1])
    
    lp.set_niter(3)
    lp.set_Q(Qlist[i])
    lp.set_B(100.)
    # START HERE
    if len(upstream_segment_list[i]) == 0:
        Qs0 = lp.k_Qs * lp.Q[0] * (1*S0)**(7/6.)
        lp.set_Qs_input_upstream(Qs0)
    i += 1
    lp.set_uplift_rate(0)
    
def i_empty(_list):
    out = []
    for i in range(len(_list)):
        if len(_list)[i] == 0:
            out.append(i)
    return out

i = 0
for lp in segments:
    if len(downstream_segment_list[i]) == 0:
        lp.set_z_bl(lp.z_ext[-1])
    i += 1

net = grlp.Network(segments)
net.get_z_lengths()
net.set_niter(3)
net.build_ID_list()
#net.set_dQ()
#segments[2].dQ[0] = 0
#segments[0].dQ[-1] = 10
#segments[1].dQ[-1] = 10
net.evolve_threshold_width_river_network(nt=100, dt=dt)

_imovie = 0
plt.figure()
for lp in segments:
    for _id in lp.downstream_segment_IDs:
        dsseg = net.list_of_LongProfile_objects[_id]
        _xjoin = [lp.x[-1], dsseg.x[0]]
        _zjoin = [lp.z[-1], dsseg.z[0]]
        plt.plot(_xjoin, _zjoin, 'k-', linewidth=4, alpha=.5)
    if len(lp.downstream_segment_IDs) == 0:
        plt.plot(lp.x_ext[1:], lp.z_ext[1:], '-', linewidth=4, alpha=.5)#, label=lp.)
    else:
        plt.plot(lp.x, lp.z, '-', linewidth=4, alpha=.5)#, label=lp.)
        
plt.ylim(-25,200)
# plt.xlim(0,11000)
plt.tight_layout()
plt.text(4000, 150, "Start.", fontsize=26, fontweight='bold')
plt.draw()
#plt.savefig('/home/awickert/Downloads/RiverNetMovie/RiverNet4seg' + '%03d' %_imovie + '.png')
_imovie += 1
plt.pause(0.5)
#plt.legend()

#don't need to do this now.

# net.list_of_LongProfile_objects[4].set_Qs_input_upstream(
#                                  2 * net.list_of_LongProfile_objects[4].Q_s_0)

# for ti in range(10):
#     net.evolve_threshold_width_river_network(nt=10, dt=dt)
#     #plt.figure()
#     plt.cla()
#     for lp in segments:
#         for _id in lp.downstream_segment_IDs:
#             dsseg = net.list_of_LongProfile_objects[_id]
#             _xjoin = [lp.x[-1], dsseg.x[0]]
#             _zjoin = [lp.z[-1], dsseg.z[0]]
#             plt.plot(_xjoin, _zjoin, 'k-', linewidth=4, alpha=.5)
#         if len(lp.downstream_segment_IDs) == 0:
#             plt.plot(lp.x_ext[1:], lp.z_ext[1:], '-', linewidth=4, alpha=.5)#, label=lp.)
#         else:
#             plt.plot(lp.x, lp.z, '-', linewidth=4, alpha=.5)#, label=lp.)
#     plt.ylim(-25,200)
#     # plt.xlim(0,11000)
#     plt.tight_layout()
#     plt.text(4000, 150, "Doubling sediment supply:\nblue branch\n$\Delta t =$ 10 years", fontsize=16, fontweight='bold')
#     plt.draw()
#     #plt.savefig('/home/awickert/Downloads/RiverNetMovie/RiverNet4seg' + '%03d' %_imovie + '.png')
#     _imovie += 1
#     plt.pause(0.5)
#     #plt.legend()

# net.list_of_LongProfile_objects[-1].set_z_bl(-20)
# for ti in range(10):
#     net.evolve_threshold_width_river_network(nt=2, dt=dt)
#     #plt.figure()
#     plt.cla()
#     for lp in segments:
#         for _id in lp.downstream_segment_IDs:
#             dsseg = net.list_of_LongProfile_objects[_id]
#             _xjoin = [lp.x[-1], dsseg.x[0]]
#             _zjoin = [lp.z[-1], dsseg.z[0]]
#             plt.plot(_xjoin, _zjoin, 'k-', linewidth=4, alpha=.5)
#         if len(lp.downstream_segment_IDs) == 0:
#             plt.plot(lp.x_ext[1:], lp.z_ext[1:], '-', linewidth=4, alpha=.5)#, label=lp.)
#         else:
#             plt.plot(lp.x, lp.z, '-', linewidth=4, alpha=.5)#, label=lp.)
#     plt.ylim(-25,200)
#     # plt.xlim(0,11000)
#     plt.text(4000, 150, "Base-level fall: 20 m\nBlue branch still 2x sed\n$\Delta t =$ 2 years", fontsize=16, fontweight='bold')
#     plt.tight_layout()
#     plt.draw()
#     #plt.savefig('/home/awickert/Downloads/RiverNetMovie/RiverNet4seg' + '%03d' %_imovie + '.png')
#     _imovie += 1
#     plt.pause(0.5)
#     #plt.legend()


# for lp in segments:
#     lp.compute_Q_s()
#     print(lp.Q_s)


In [ ]:
lp = net.list_of_LongProfile_objects[2]
lp.z_ext